In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pydicom
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from PIL import Image
from imblearn.over_sampling import SMOTE
from tensorflow.keras.applications.efficientnet import EfficientNetB0

import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

/Users/johngalvin/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Approach

1. Pretrain a model to classify AD/No AD brain scans on a different dataset
2. Use the pretrained model to generate embeddings (can be for an arbitrary number of images) for the ADNI training/test set
3. Use Pooling to combine the embeddings for a given person into a fixed shape
4. Concatenate the pooled vector with the clinical/genetic data
5. Train model

We end up with 2 models total.

## Fine-tune EfficientNetB0 on OASIS dataset

Note: We must retrain a portion of the trunk since we will remove the head we attach here to generate embeddings later on.

In [2]:
NUM_CLASSES = 1
IMG_SIZE = (224,224) # Expected size for EfficientNetB0
NUM_EPOCHS = 15
BATCH_SIZE = 64
LR = 0.0001

In [3]:
# Load and convert images to .jpeg
img_dir = "/Users/johngalvin/Desktop/OASIS/0/"

for file in os.listdir(img_dir):
    if file.endswith(".JPG") or file.endswith(".jpg"):
        img = Image.open(img_dir + file)
        file_name, file_ext = os.path.splitext(file)
        new_name = file_name + ".jpeg"
        img.save(img_dir + new_name)
        
img_dir = "/Users/johngalvin/Desktop/OASIS/1/"

for file in os.listdir(img_dir):
    if file.endswith(".JPG") or file.endswith(".jpg"):
        img = Image.open(img_dir + file)
        file_name, file_ext = os.path.splitext(file)
        new_name = file_name + ".jpeg"
        img.save(img_dir + new_name)

In [4]:
# Delete the .JPG and.jpg files
img_dir = "/Users/johngalvin/Desktop/OASIS/0/"
for file in os.listdir(img_dir):
    if file.endswith(".JPG") or file.endswith(".jpg"):
        path_to_file = os.path.join("/Users/johngalvin/Desktop/OASIS/0/", file)
        os.remove(path_to_file)
        
img_dir = "/Users/johngalvin/Desktop/OASIS/1/"
for file in os.listdir(img_dir):
    if file.endswith(".JPG") or file.endswith(".jpg"):
        path_to_file = os.path.join("/Users/johngalvin/Desktop/OASIS/1/", file)
        os.remove(path_to_file)

In [5]:
targets = []
arrays = []

img_dir = "/Users/johngalvin/Desktop/OASIS/0/"
for file in os.listdir(img_dir):
    fpath = os.path.join("/Users/johngalvin/Desktop/OASIS/0/", file)
    img = Image.open(fpath).convert("L")  # Convert the image to grayscale
    resized_image = img.resize((224, 224), Image.BILINEAR)  # Resize the image
    resized_array = np.expand_dims(np.array(resized_image, dtype=np.uint8), axis=-1)  # Convert to NumPy array and add channel dimension
    targets.append(0)
    arrays.append(resized_array)

In [6]:
img_dir = "/Users/johngalvin/Desktop/OASIS/1/"
for file in os.listdir(img_dir):
    fpath = os.path.join("/Users/johngalvin/Desktop/OASIS/1/", file)
    img = Image.open(fpath).convert("L")  # Convert the image to grayscale
    resized_image = img.resize((224, 224), Image.BILINEAR)  # Resize the image
    resized_array = np.expand_dims(np.array(resized_image, dtype=np.uint8), axis=-1)  # Convert to NumPy array and add channel dimension
    targets.append(1)
    arrays.append(resized_array)
    
X = np.array(arrays)
y = np.array(targets)

In [7]:
# Balance positive and negative class
y = y[-y.sum()*2:]
X = X[-y.sum()*2:]

In [8]:
# Split data
X_train_set, X_test, y_train_set, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reduce size to fit in memory
X_train = X_train_set[:5000]
y_train = y_train_set[:5000]
X_val = X_train_set[5000:6000]
y_val = y_train_set[5000:6000]
X_test = X_test[:1000]
y_test = y_test[:1000]

# Scale data
scaler = MinMaxScaler()

num_samples, height, width, channels = X_train.shape
X_train_reshaped = X_train.reshape(num_samples, -1)
X_train_scaled_2d = scaler.fit_transform(X_train_reshaped)
X_train_scaled = X_train_scaled_2d.reshape(num_samples, height, width, channels)

num_samples, height, width, channels = X_val.shape
X_val_reshaped = X_val.reshape(num_samples, -1)
X_val_scaled_2d = scaler.transform(X_val_reshaped)
X_val_scaled = X_val_scaled_2d.reshape(num_samples, height, width, channels)

num_samples, height, width, channels = X_test.shape
X_test_reshaped = X_test.reshape(num_samples, -1)
X_test_scaled_2d = scaler.transform(X_test_reshaped)
X_test_scaled = X_test_scaled_2d.reshape(num_samples, height, width, channels)

In [9]:
# Add channels (EfficientNetB0 expects 3 channels)
X_train_rgb = np.repeat(X_train_scaled, 3, axis=-1)
X_val_rgb = np.repeat(X_val_scaled, 3, axis=-1)
X_test_rgb = np.repeat(X_test_scaled, 3, axis=-1)

In [10]:
# Freeze trunk and train top
inputs = tf.keras.layers.Input(shape=IMG_SIZE + (3,))

efficient_net_base_model = EfficientNetB0(include_top=False,
                                          input_shape=IMG_SIZE + (3,),
                                          weights="imagenet")

x = efficient_net_base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.3
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="sigmoid", name="pred")(x)
embedding_model = tf.keras.Model(inputs, outputs, name="EfficientNet")

for layer in embedding_model.layers[-20:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

embedding_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                        optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
                        metrics=["accuracy"])

Metal device set to: Apple M2 Pro


2023-10-07 13:58:54.297796: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-07 13:58:54.297839: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
tf.config.run_functions_eagerly(True)
embedding_model_history = embedding_model.fit(X_train_rgb,
                                              y_train,
                                              validation_data=[X_val_rgb, y_val],
                                              epochs=NUM_EPOCHS,
                                              batch_size=BATCH_SIZE)

/Users/johngalvin/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15


2023-10-07 13:59:07.204724: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/johngalvin/miniforge3/lib/python3.9/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


79/79 [==============================] - 161s 2s/step - loss: 0.5237 - accuracy: 0.7572 - val_loss: 0.7111 - val_accuracy: 0.5050
Epoch 2/15
79/79 [==============================] - 181s 2s/step - loss: 0.2563 - accuracy: 0.8956 - val_loss: 0.7214 - val_accuracy: 0.5050
Epoch 3/15
79/79 [==============================] - 182s 2s/step - loss: 0.1507 - accuracy: 0.9436 - val_loss: 0.7058 - val_accuracy: 0.5400
Epoch 4/15
79/79 [==============================] - 180s 2s/step - loss: 0.0931 - accuracy: 0.9680 - val_loss: 0.7128 - val_accuracy: 0.5190
Epoch 5/15
79/79 [==============================] - 180s 2s/step - loss: 0.0736 - accuracy: 0.9748 - val_loss: 0.8817 - val_accuracy: 0.4850
Epoch 6/15
79/79 [==============================] - 181s 2s/step - loss: 0.0576 - accuracy: 0.9796 - val_loss: 0.6791 - val_accuracy: 0.5740
Epoch 7/15
79/79 [==============================] - 498s 6s/step - loss: 0.0554 - accuracy: 0.9794 - val_loss: 0.7205 - val_accuracy: 0.5600
Epoch 8/15
79/79 [======

In [12]:
embedding_model.summary()

Model: "EfficientNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 avg_pool (GlobalAveragePool  (None, 1280)             0         
 ing2D)                                                          
                                                                 
 batch_normalization (BatchN  (None, 1280)             5120      
 ormalization)                                                   
                                                                 
 top_dropout (Dropout)       (None, 1280)              0         
                                                                 
 pred (Dense)                (None, 1)                

In [13]:
embedding_model.evaluate(X_test_rgb, y_test)

32/32 [==============================] - 6s 157ms/step - loss: 0.8509 - accuracy: 0.4610


[0.8508883118629456, 0.4610000252723694]

##### If this doesn't work, instead of fine-tuning, just create a fully connected network (small/simple)

In [23]:
testmodel = tf.keras.Model(embedding_model.input, embedding_model.get_layer('avg_pool').output)

In [24]:
testmodel.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 avg_pool (GlobalAveragePool  (None, 1280)             0         
 ing2D)                                                          
                                                                 
Total params: 4,049,571
Trainable params: 4,007,548
Non-trainable params: 42,023
_________________________________________________________________


## Generate embeddings for ADNI images

In [25]:
pt_ids = []
pixels = []

directory_path = '/Users/johngalvin/Downloads/ADNI 2'

# Iterate through level 2 subdirectories
for level_2_foldername in os.listdir(directory_path):
    level_2_folder_path = os.path.join(directory_path, level_2_foldername)
    
    if os.path.isdir(level_2_folder_path):
        # Iterate through DICOM files in level 5 (bottom-most level) of each level 2 folder
        for root, _, files in os.walk(level_2_folder_path):
            for file in files:
                try:
                    file_path = os.path.abspath(os.path.join(root, file))
                    
                    # Attempt to read DICOM file
                    dcm = pydicom.dcmread(file_path)
                    
                    # Check if the file has PixelData (to avoid non-image DICOM files)
                    if hasattr(dcm, 'PixelData'):
                        # Append both level 2 folder name and pixel array to the lists
                        pt_ids.append(file[5:15])
                        pixels.append(dcm.pixel_array)
                except Exception as e:
                    # Handle exceptions (e.g., files without 'TransferSyntaxUID')
                    print(f"Error processing file {file_path}: {e}")

# Create a DataFrame from the lists
mri_df = pd.DataFrame({'PTID': pt_ids, 'Pixel Array': pixels})

Error processing file /Users/johngalvin/Downloads/ADNI 2/068_S_0473/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/068_S_0473/MPRAGE/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/032_S_0677/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/032_S_0677/MPRAGE/.DS_Store: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
Error processing file /Users/johngalvin/Downloads/ADNI 2/032_S_0677/MPRAGE/2016-07-22_09_23_31.0/.DS_Store: File is missing DICOM File Met

In [26]:
# Resize image arrays with Bilinear Interpolation
resized_arrays = []

for val in mri_df["Pixel Array"]:
    image = Image.fromarray(val, mode='L')
    resized_image = image.resize((224, 224), Image.BILINEAR)
    resized_array = np.expand_dims(np.array(resized_image, dtype=np.uint8), axis=-1) #TF expects channel dim
    resized_arrays.append(resized_array)
    
mri_df["Pixel Array"] = resized_arrays

In [37]:
# Convert shape to (num_samples, 224, 224, 3)
pixels = np.array(mri_df["Pixel Array"])
pixels = np.stack(pixels, axis=0)
pixels = np.repeat(pixels, 3, axis=-1)

In [45]:
# Generate embeddings
latent_reps = []

for i in range(len(pixels)):
    latent_reps.append(testmodel(np.expand_dims(pixels[i], axis=0)))

KeyboardInterrupt: 

In [ ]:
mri_df["Latent Rep"] = latent_reps

## Combine embeddings for a given subject into a single vector

In [ ]:
def mean_pooling(vectors):
    return np.mean(vectors, axis=0)

## Merge Clinical/Genetic Data with MRI Arrays

In [2]:
mf_hist = pd.read_csv('../data/clinical_training_data_with_medhist_famhist.csv')

# Pre-process Clinical/Genetic

In [9]:
# Handle Nan
df["Family_History_of_AD"] = df["Family_History_of_AD"].fillna(0)
df["Family_History_of_Dementia"] = df["Family_History_of_Dementia"].fillna(0)

# For converting categorical variables to ints
label_encoder = LabelEncoder()
scaler = MinMaxScaler()

# Split features / target
X = df.drop(columns=['AD_dx_in_5_yrs'])
y = df['AD_dx_in_5_yrs']

# Encode features
X["Diagnosis_at_Baseline"] = label_encoder.fit_transform(X["Diagnosis_at_Baseline"])
X["Gender"] = label_encoder.fit_transform(X["Gender"])
X["Ethnicity"] = label_encoder.fit_transform(X["Ethnicity"])
X["Race"] = label_encoder.fit_transform(X["Race"])